In [1]:
# $ 0 u l $ h i f t e r
import pandas as pd
import numpy as np
import os
from os import listdir
import warnings
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
train_path = "../dataset/train/"
subfolders = os.listdir(train_path)
subfolders = subfolders[1:]
test_path = "../dataset/test_stg1/"
test_files = os.listdir(test_path)
subfolders
test_files[:10]

['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

['img_00005.jpg',
 'img_00007.jpg',
 'img_00009.jpg',
 'img_00018.jpg',
 'img_00027.jpg',
 'img_00030.jpg',
 'img_00040.jpg',
 'img_00046.jpg',
 'img_00053.jpg',
 'img_00071.jpg']

In [3]:
def load_images_train(path):
    subfolders = os.listdir(path)
    images = []
    category = []
    for folder in subfolders[1:]:
        entirePath = path + folder + "/"
        imageList = os.listdir(entirePath)
        for image in imageList:
            img = entirePath + image
            images.append(img)
            category.append(folder)
    return images, category

In [4]:
images, category = load_images_train(train_path)
images[:10]
category[:10]

['../dataset/train/ALB/img_00003.jpg',
 '../dataset/train/ALB/img_00010.jpg',
 '../dataset/train/ALB/img_00012.jpg',
 '../dataset/train/ALB/img_00015.jpg',
 '../dataset/train/ALB/img_00019.jpg',
 '../dataset/train/ALB/img_00020.jpg',
 '../dataset/train/ALB/img_00029.jpg',
 '../dataset/train/ALB/img_00032.jpg',
 '../dataset/train/ALB/img_00037.jpg',
 '../dataset/train/ALB/img_00038.jpg']

['ALB', 'ALB', 'ALB', 'ALB', 'ALB', 'ALB', 'ALB', 'ALB', 'ALB', 'ALB']

In [5]:
num_category = len(np.unique(category))
num_category

8

In [6]:
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
lbc = LabelEncoder()
encoded_category = lbc.fit_transform(category)
category_ohe = to_categorical(encoded_category, num_classes=num_category)
category_ohe

Using TensorFlow backend.


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [7]:
from keras.preprocessing.image import ImageDataGenerator
DIM = (128, 128)
img_gen = ImageDataGenerator(samplewise_center=True,
                            samplewise_std_normalization=True,
                            horizontal_flip=True,
                            vertical_flip=False,
                            height_shift_range=0.05,
                            width_shift_range=0.1,
                            rotation_range=5,
                            shear_range=0.01,
                            fill_mode="reflect",
                            zoom_range=0.15)

In [8]:
df = pd.DataFrame(index=None, columns=["path", "category"])
df["path"] = images
df["category"] = list(category_ohe)
df.head()

,path,category
0,../dataset/train/ALB/img_00003.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,../dataset/train/ALB/img_00010.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,../dataset/train/ALB/img_00012.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,../dataset/train/ALB/img_00015.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,../dataset/train/ALB/img_00019.jpg,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [9]:
def flow_from_dataframe(img_gen, df, x_col, y_col, **args):
    base_dir = os.path.dirname(df[x_col].values[0])
    df_gen = img_gen.flow_from_directory(base_dir, class_mode="sparse", **args)
    df_gen.filenames = df[x_col].values
    df_gen.classes = np.stack(df[y_col].values)
    df_gen.samples = df.shape[0]
    df_gen.n = df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = ""
    return df_gen

In [10]:
from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.25, random_state=4)
train_df.shape
valid_df.shape

(2832, 2)

(945, 2)

In [11]:
train_gen = flow_from_dataframe(img_gen, train_df, "path", "category", target_size=DIM, batch_size=128)
valid_gen = flow_from_dataframe(img_gen, valid_df, "path", "category", target_size=DIM, batch_size=256)
test_x, test_y = next(flow_from_dataframe(img_gen, valid_df, "path", "category", target_size=DIM, batch_size=1024))

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [12]:
train_x, train_y = next(train_gen)
train_x.shape
train_y.shape
img_dim = train_x.shape[1:]
img_dim

(128, 128, 128, 3)

(128, 8)

(128, 128, 3)

In [13]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import fbeta_score

In [14]:
inp_tens = Input(shape=img_dim)
base_model = VGG16(include_top=False, input_shape=img_dim)
bn = BatchNormalization()(inp_tens)
x = base_model(bn)
x = Flatten()(x)
output = Dense(num_category, activation="softmax")(x)
model = Model(inp_tens, output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 3)       12        
_________________________________________________________________
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 65544     
Total params: 14,780,244
Trainable params: 14,780,238
Non-trainable params: 6
_________________________________________________________________


In [15]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History
from keras.optimizers import Adam

In [16]:
history = History()
callbacks = [history, EarlyStopping(monitor="val_loss", patience=3, verbose=1, min_delta=1e-4), ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=1,
                                                                                                                 cooldown=0, min_lr=1e-7, verbose=1),
            ModelCheckpoint(filepath="weights.best.hd5", verbose=1, save_best_only=True, save_weights_only=True, mode="auto")]

In [17]:
model.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=["accuracy"])

In [18]:
batch_size = 512
steps_per_epoch = len(train_df) / batch_size
steps_per_epoch

5.53125

In [19]:
# model.fit_generator(train_gen, steps_per_epoch=steps_per_epoch, validation_data=(test_x, test_y), epochs=15, callbacks=callbacks, use_multiprocessing=True, workers=-1)

In [20]:
model.load_weights("weights.best.hd5")

In [21]:
from glob import glob
test_files_path = glob(test_path + "*jpg", recursive=True)
test_files_path[:5]
submissionDf = pd.DataFrame(index=None, columns=["path", "image"])
submissionDf["path"] = test_files_path
test_files_path = [x.replace("\\", "/", 2) for x in test_files_path]
path_split = [x.split("/") for x in test_files_path]
image_name = [x[-1] for x in path_split]
submissionDf["path"] = test_files_path
submissionDf["image"] = image_name
submissionDf.head()

['../dataset/test_stg1\\img_00005.jpg',
 '../dataset/test_stg1\\img_00007.jpg',
 '../dataset/test_stg1\\img_00009.jpg',
 '../dataset/test_stg1\\img_00018.jpg',
 '../dataset/test_stg1\\img_00027.jpg']

,path,image
0,../dataset/test_stg1/img_00005.jpg,img_00005.jpg
1,../dataset/test_stg1/img_00007.jpg,img_00007.jpg
2,../dataset/test_stg1/img_00009.jpg,img_00009.jpg
3,../dataset/test_stg1/img_00018.jpg,img_00018.jpg
4,../dataset/test_stg1/img_00027.jpg,img_00027.jpg


In [34]:
test_gen = flow_from_dataframe(img_gen, submissionDf, x_col="path", y_col="image", target_size=DIM, batch_size=256)

Found 0 images belonging to 0 classes.


In [35]:
pred = model.predict_generator(test_gen, verbose=1, steps=4)

4/4 [==============================] - 208s 52s/step


In [45]:
len(pred)

1000

In [46]:
unique_labels = np.unique(category)
lbc.fit_transform(unique_labels)
for i, label in enumerate(list(unique_labels)):
    (i, label)
    submissionDf[label] = pred[:,i]

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

(0, 'ALB')

(1, 'BET')

(2, 'DOL')

(3, 'LAG')

(4, 'NoF')

(5, 'OTHER')

(6, 'SHARK')

(7, 'YFT')

In [48]:
submissionDf.head()
submissionDf.drop("label", axis=1, inplace=True)
submissionDf.head()

,path,image,label,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,../dataset/test_stg1/img_00005.jpg,img_00005.jpg,0.083080,0.390138,0.076374,0.119023,0.014287,0.107464,0.046356,0.163277,0.083080
1,../dataset/test_stg1/img_00007.jpg,img_00007.jpg,0.206900,0.378268,0.044148,0.044855,0.032445,0.053394,0.138160,0.101830,0.206900
2,../dataset/test_stg1/img_00009.jpg,img_00009.jpg,0.052434,0.719259,0.032534,0.009033,0.013369,0.066399,0.091408,0.015564,0.052434
3,../dataset/test_stg1/img_00018.jpg,img_00018.jpg,0.150078,0.602932,0.039236,0.030805,0.012470,0.043484,0.100047,0.020949,0.150078
4,../dataset/test_stg1/img_00027.jpg,img_00027.jpg,0.062509,0.160896,0.039787,0.042556,0.012302,0.561129,0.064793,0.056028,0.062509


,path,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,../dataset/test_stg1/img_00005.jpg,img_00005.jpg,0.390138,0.076374,0.119023,0.014287,0.107464,0.046356,0.163277,0.083080
1,../dataset/test_stg1/img_00007.jpg,img_00007.jpg,0.378268,0.044148,0.044855,0.032445,0.053394,0.138160,0.101830,0.206900
2,../dataset/test_stg1/img_00009.jpg,img_00009.jpg,0.719259,0.032534,0.009033,0.013369,0.066399,0.091408,0.015564,0.052434
3,../dataset/test_stg1/img_00018.jpg,img_00018.jpg,0.602932,0.039236,0.030805,0.012470,0.043484,0.100047,0.020949,0.150078
4,../dataset/test_stg1/img_00027.jpg,img_00027.jpg,0.160896,0.039787,0.042556,0.012302,0.561129,0.064793,0.056028,0.062509


In [49]:
submissionDf.to_csv("predictions_test_stage_1.csv")